# legacy code
y_columns_names = ['Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции']
X = df.drop(y_columns_names, axis=1)
X

y = df[y_columns_names]
y

nlp = spacy.load('ru_core_news_sm', disable=['tagger', 'parser', 'ner', 'entity_linker', 'entity_ruler', 'textcat', 'tokenizer', 'morphologizer', 'attribute_ruler', 'senter', 'sentencizer', 'transformer'])
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 1:
        return ' '.join(txt)
    return txt[0]

# normalize data

In [1]:
import pandas as pd
import numpy
import re
from collections import defaultdict
import spacy
import logging

In [2]:
df = pd.read_excel('датасет 15072022.xlsx').replace('_', ' ')

In [3]:
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'tagger', 'ner'])

def normalize(doc):
    doc = nlp(str(doc).lower())
    lemmatized = list()
    for token in doc:
        if not token.is_stop and not token.is_punct:
            lemmatized.append(token.lemma_.strip())
    return " ".join(lemmatized)

In [26]:
type(df['Общее наименование продукции'])

pandas.core.series.Series

In [4]:
norm1 = df['Общее наименование продукции'].apply(normalize)
print('PROGRESS 33% ...')
norm2 = df['Группа продукции'].apply(normalize)
print('PROGRESS 66% ...')
norm3 = df['Технические регламенты'].apply(normalize)

# df['CLEAN Общее наименование продукции'].apply(normalize)
# df['CLEAN Группа продукции'] = df['Группа продукции'].apply(normalize)
# df['CLEAN Технические регламенты'] = df['Технические регламенты'].apply(normalize)

PROGRESS 33% ...
PROGRESS 66% ...


In [5]:
df_clean = pd.DataFrame({'Общее наименование продукции': norm1, 'Группа продукции': norm2, 'Технические регламенты': norm3})
# df_clean = df[['CLEAN Общее наименование продукции', 'CLEAN Группа продукции', 'CLEAN Технические регламенты']].copy()
df_clean = df_clean.dropna()
df_clean.shape

(66753, 3)

In [17]:
df_clean

,Общее наименование продукции,Группа продукции,Технические регламенты
0,проходка кабельный модульный универсальный огн...,узел пересечение противопожарный преграда кабе...,тр еаэс 043/2017 требование средство обеспечен...
1,ворота противопожарный металлический распашной...,противопожарный окно дверь дверь шахта лифт но...,тр еаэс 043/2017 требование средство обеспечен...
2,клапаны противопожарный марка кпв.01(ei60 прям...,клапан противопожарный нормально открытый клап...,тр еаэс 043/2017 требование средство обеспечен...
3,вентилятор крышный радиальный дымоудаления вых...,вытяжной вентилятор,тр еаэс 043/2017 требование средство обеспечен...
4,вентилятор радиальный дымоудаления тип вр-80 7...,вытяжной вентилятор,тр еаэс 043/2017 требование средство обеспечен...
...,...,...,...
66748,блок пробоотборный обогреваемый,электрический электрооборудование неэлектричес...,тр тс 012/2011 безопасность оборудование работ...
66749,система отбор проба природный газ,электрический электрооборудование неэлектричес...,тр тс 012/2011 безопасность оборудование работ...
66750,фильтр,электрический электрооборудование неэлектричес...,тр тс 012/2011 безопасность оборудование работ...
66751,элемент трубопровод,элемент оборудование сборочный единица комплек...,тр тс 032/2013 безопасность оборудование работ...


In [6]:
from gensim.models.phrases import Phrases, Phraser
sent0 = [row.split() for row in df_clean['Общее наименование продукции']]
sent1 = [row.split() for row in df_clean['Группа продукции']]
sent2 = [row.split() for row in df_clean['Технические регламенты']]
sent = []
for i in range(len(sent1)):
    sent.append(sent0[i] + sent1[i] + sent2[i])

# Word2Vec

In [7]:
phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

48493

In [8]:
import multiprocessing
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(min_count=5, window=2, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20, workers=cores-1)

In [9]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [10]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(19822050, 52847880)

In [11]:
w2v_model.init_sims(replace=True)

/tmp/ipykernel_7719/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


# optional visualization

In [12]:
w2v_model.wv.similarity("адаптер", 'асинхронный')

0.1950113

In [13]:
w2v_model.wv.most_similar(positive=["вентилятор"])

[('осевой', 0.7823437452316284),
 ('вентилятор_вентилятор', 0.7431402206420898),
 ('крышный_радиальный', 0.7173740863800049),
 ('радиальный', 0.7078759670257568),
 ('проветривание', 0.7050995826721191),
 ('радиальный_дымоудаления', 0.6305767297744751),
 ('дымоудаления', 0.6229865550994873),
 ('аксипал', 0.6215609312057495),
 ('тепловентилятор', 0.6091592311859131),
 ('пылеподавление', 0.6007325649261475)]

# export vectors

In [14]:
print(w2v_model.wv.vectors.shape[1], w2v_model.wv.vectors.shape[0])

100 10109


In [15]:
with open("Output.txt", "w") as text_file:
    text_file.write(str(w2v_model.wv.vectors.shape[0]) + ' ' + str(w2v_model.wv.vectors.shape[1]) + '\r\n')
    for i in range(len(w2v_model.wv.index_to_key)):
        final = ''
        for j in range(len(w2v_model.wv.vectors[i])):
            final += str(w2v_model.wv.vectors[i][j]) + ' '
        text_file.write(str(w2v_model.wv.index_to_key[i]) + ' ' + final + '\r\n')

# save normalized name as an extra column

In [18]:
df['NORMALIZED'] = df_clean['Общее наименование продукции']
df

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя,NORMALIZED
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ,проходка кабельный модульный универсальный огн...
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ,ворота противопожарный металлический распашной...
2,2112 Продукция,8415900009; 8415900009; 8415900009; 8415900009...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Клапаны противопожарные нормально открытые, кл...",Клапаны противопожарные марки КПВ.01(EI60) пря...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,клапаны противопожарный марка кпв.01(ei60 прям...
3,50677 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы крышные радиальные дымоудаления с ...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,вентилятор крышный радиальный дымоудаления вых...
4,51009 Продукция,8414594000,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Вытяжные вентиляторы,Вентиляторы радиальные дымоудаления типов ВР-8...,ИЛ Мопалия Миддендорфа,ООО Малая белая цапля1,РОССИЯ 162833 ДОЛОЦКОЕ ВОЛОГОДСКАЯ ОБЛАСТЬ,Phelsuma serraticauda 1 LTD,ТУРЦИЯ,TR 49400 Muş Muratkolu,вентилятор радиальный дымоудаления тип вр-80 7...
...,...,...,...,...,...,...,...,...,...,...,...,...
66748,13 Продукция,NaN,ТР ТС 012/2011 О безопасности оборудования для...,Электрическое (электрооборудование) и неэлектр...,Блок пробоотборный обогреваемый,ИЛ Пестрая афродита,ООО Азиатская золотая кошка 1,РОССИЯ 679157 АУР ЕВРЕЙСКАЯ АВТОНОМНАЯ ОБЛАСТЬ,Catopuma temmincki 1 LTD,АЛЖИР,DZ 03000 L.Aghouat Laghouat Air,блок пробоотборный обогреваемый
66749,29 Продукция,NaN,ТР ТС 012/2011 О безопасности оборудования для...,Электрическое (электрооборудование) и неэлектр...,Система отбора проб природного газа,ИЛ Пестрая афродита,ООО Азиатская золотая кошка 1,РОССИЯ 679157 АУР ЕВРЕЙСКАЯ АВТОНОМНАЯ ОБЛАСТЬ,Catopuma temmincki 1 LTD,АЛЖИР,DZ 03000 L.Aghouat Laghouat Air,система отбор проба природный газ
66750,41883 Продукция,7311009100,ТР ТС 012/2011 О безопасности оборудования для...,Электрическое (электрооборудование) и неэлектр...,Фильтры,ИЛ Рак-богомол,ООО Азиатская золотая кошка 1,РОССИЯ 679157 АУР ЕВРЕЙСКАЯ АВТОНОМНАЯ ОБЛАСТЬ,Catopuma temmincki 1 LTD,АЛЖИР,DZ 03000 L.Aghouat Laghouat Air,фильтр
66751,59784 Продукция,7307998009; 7307910000,"ТР ТС 032/2013 О безопасности оборудования, ра...",Элементы оборудования (сборочные единицы) и ко...,Элементы трубопроводов,ИЛ Пестрая афродита,ООО Азиатская золотая кошка 1,РОССИЯ 679157 АУР ЕВРЕЙСКАЯ АВТОНОМНАЯ ОБЛАСТЬ,Catopuma temmincki 1 LTD,АЛЖИР,DZ 03000 L.Aghouat Laghouat Air,элемент трубопровод


In [19]:
df.to_excel("normalized.xlsx")